In [1]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output, State
import plotly.graph_objs as go
import dash_bootstrap_components as dbc
import numpy as np

In [2]:
# Read the Excel file
df = pd.read_excel("staff.xlsx")

# Read the project types
df_types = pd.read_excel("types.xlsx")

# Load department data
df_departments = pd.read_excel("departments.xlsx")

# Create dropdown menu options from unique project names
project_options = [{'label': project, 'value': project} for project in df['Project'].unique()]

# Create dropdown menu options for project types
type_options = [{'label': project_type, 'value': project_type} for project_type in df_types['Type'].unique()]

# Create dropdown menu options from unique departments including 'Select All'
department_options = [{'label': 'Select All', 'value': 'all'}] + [{'label': department, 'value': department} for department in df_departments['Department'].dropna().unique()]

# Add "Other" option for null department
department_options.append({'label': 'Other', 'value': 'Other'})

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [3]:
# Define the layout of the app
app.layout = html.Div([
    html.H1("Deployment Plan"),
    html.Button("Export to PDF", id="export-button"),
    html.Div(id="export-prompt"),  # Placeholder for export feedback
    dbc.Button("New Project", id="open", color="primary", className="mr-1"),
    dbc.Modal(
        [
            dbc.ModalHeader("New Project Details"),
            dbc.ModalBody(
                [
                    dcc.Input(id='type-input', type='text', placeholder='Type of Project'),
                    dcc.Input(id='duration-input', type='text', placeholder='Duration of Project')
                ]
            ),
            dbc.ModalFooter(
                [
                    dbc.Button("Close", id="close", className="ml-auto"),
                    dbc.Button("Save Changes", id="save-changes", className="ml-1")
                ]
            ),
        ],
        id="modal",
        size='lg',
        centered=True,
        backdrop='static'
    ),
    html.Div([
        html.Label('Type of Project:'),
        dcc.Dropdown(
            id='type-dropdown',
            options=type_options,
            value=type_options[0]['value'] if type_options else None  # Default value
        )
    ]),
    html.Div([
        html.Label('Select Department:'),
        dcc.Dropdown(
            id='department-dropdown',
            options=department_options,
            value='all'  # Default value
        ),
    ]),
    dcc.Dropdown(
        id='project-dropdown',
        options=project_options,
        value=project_options[0]['value']  # Default value
    ),
    dcc.Dropdown(
        id='job-dropdown',
        multi=True,
    ),
    dcc.Graph(id='deployment-plan'),
    html.Div(id='selected-project-type')  # Show the selected project type
])


In [4]:
# Define callback to update job dropdown options based on selected project, department, and job titles

@app.callback(
    Output('job-dropdown', 'options'),
    [Input('project-dropdown', 'value'),
     Input('department-dropdown', 'value'),
     Input('job-dropdown', 'value')]
)
def update_job_dropdown(selected_project, selected_department, selected_jobs):
    try:
        filtered_jobs = None  # Initialize filtered_jobs to avoid errors

        if selected_department == 'Select All':
            filtered_jobs = df_departments['Job'].unique()
        else:
            filtered_jobs = df_departments[df_departments['Department'] == selected_department]['Job'].unique()

        # Preserve 'Select All' option if not already selected
        if selected_jobs != 'all':
            filtered_jobs = np.insert(filtered_jobs, 0, 'all')

        job_options = [{'label': job, 'value': job} for job in filtered_jobs]

        return job_options

    except Exception as e:
        print(f"Error in update_job_dropdown: {e}")
        return []


In [5]:
# Define callback to update the deployment plan based on selected project, department, and job titles

@app.callback(
    Output('deployment-plan', 'figure'),
    [Input('project-dropdown', 'value'),
     Input('job-dropdown', 'value'),
     Input('department-dropdown', 'value')]
)
def update_plan(selected_project, selected_jobs, selected_department):

    # Handle empty inputs gracefully
    if not all([selected_project, selected_jobs, selected_department]):
        return {'data': [], 'layout': {}}

    # Perform filtering only once for consistency
    filtered_data = df[(df['Project'] == selected_project) &
                       (df['Job'].isin(selected_jobs)) &
                       ((selected_department == 'Select All') |
                        (df['Job'].isin(df_departments[df_departments['Department'] == selected_department]['Job'])))]

    if filtered_data.empty:
        return {'data': [], 'layout': {}}

    # Extract necessary data for the heatmap
    unique_jobs = filtered_data['Job'].unique()
    months = sorted(filtered_data['Month'].unique())

    staff_matrix = []
    for job in unique_jobs:
        job_data = filtered_data[filtered_data['Job'] == job]
        staff_count = [job_data[job_data['Month'] == month].shape[0] for month in months]
        staff_matrix.append(staff_count)

    max_value = max([item for sublist in staff_matrix for item in sublist])

    # Create the heatmap figure
    fig = go.Figure(data=[
        go.Heatmap(
            z=staff_matrix,
            x=months,
            y=unique_jobs,
            colorscale='Viridis',
            showscale=True,
            zmin=1,
            zmax=max_value,
            zmid=0,
            colorbar=dict(tickmode='array', tickvals=list(range(1, max_value + 1)), ticktext=list(range(1, max_value + 1)))
        )
    ], layout=go.Layout(
        title=f"Deployment Plan for {selected_project}",
        xaxis=dict(title='Months'),
        yaxis=dict(title='Job Titles'),
    ))

    return fig

In [6]:
# New callback for the modal popup
@app.callback(
    Output("modal", "is_open"),
    [Input("open", "n_clicks"), Input("close", "n_clicks"), Input("save-changes", "n_clicks")],
    [dash.dependencies.State("modal", "is_open")],
)
def toggle_modal(n1, n2, n3, is_open):
    if n1 or n2 or n3:
        return not is_open
    return is_open


In [7]:
# Callback to show the project type for the selected project
@app.callback(
    Output('selected-project-type', 'children'),
    [Input('project-dropdown', 'value')]
)
def display_project_type(selected_project):
    # Get the type of the selected project
    project_type = df_types[df_types['Project'] == selected_project]['Type'].values[0]
    return f'Type of Project: {project_type}'


In [8]:
# Import and Function to Print PDF

import pdfkit

def create_pdf_content(figure, project_type):
    html_content = """
    <h1>Deployment Plan for {project_type}</h1>
    <div>{figure}</div>
    """.format(project_type=project_type, figure=figure.to_html(full_html=False, include_plotlyjs=False))

    return html_content

In [9]:
# Callback to export/print to pdf

@app.callback(
    Output("export-prompt", "children"),
    Input("export-button", "n_clicks"),
    State("deployment-plan", "figure"),  # Access the figure object
    State("selected-project-type", "children"),
    prevent_initial_call=True
)
def export_to_pdf(n_clicks, figure, project_type):
    if n_clicks:
        # Generate PDF content using the figure object
        pdf_content = create_pdf_content(figure=figure, project_type=project_type)

        # Use pdfkit to generate the PDF
        pdfkit.from_string(pdf_content, "deployment_plan.pdf")

        return "Deployment plan exported successfully!"
    else:
        return ""

In [10]:
if __name__ == '__main__':
    app.run_server(debug=True)